# Parquet to Dask
load a parquet dataset into as dask cluster

Once the client connects to the dask cluster we can check for package consistency by calling `dask_client.get_versions(check=True)`.  Scheduler and worker
nodes are guaranteed to have the same package versions in `mlrun` since mismatches would likely cause serialization failures.  The client (in this notebook or in some other process) may have different versions and this mismatch is **less likely** to cause issues.  The following installs can be performed to ensure this mismatch becomes **unlikely** (as of this writing). **This may need to be done periodically as the dockerfiles refresh, unless you specifiy an image tag.**

    !python -m pip install --upgrade pip
    !python -m pip uninstall -y cmake
    !python -m pip install cmake
    !python -m pip install scikit-build

    !python -m pip install lz4
    !python -m pip install git+https://github.com/Blosc/python-blosc.git@v1.8.3
    !python -m pip install cloudpickle==1.2.2

    !python -m pip install numpy==1.17.4

## clean up
delete stale pods

In [40]:
# !mlrun clean -p -r

In [41]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [42]:
# replace with your app node ip
mlrun.mlconf.remote_host = '18.223.249.136' 

## parameters


In [43]:
DESCRIPTION        = 'load parquet dataset into a dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                           # custom image
JOB_KIND           = 'dask'                                           

TASK_NAME          = 'user-task-parq-to-dask'                         # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'
YAML_PATH          = '/User/repos/demos/dask/yaml'

DATA_PATH          = '/User/repos/demos/dask/dataset/partitions'
DOWNSAMPLE         = 0.01
PARTITION_COLS     = ['Year', 'Month']
PARTITION_COLS     = None

DASK_SHARDS        = 8
DASK_THREADS_PER   = 8
MEMORY_LIMIT       = '5GB'

DASK_KEY           = 'airlines'

## load and configure function

In [44]:
FUNC_PY   = '/User/repos/demos/dask/code/parquet-to-dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/parquet-to-dask.yaml'

HANDLER   = 'parquet_to_dask'

#### some of the following cells can be commented out if this notebook has already been run.

In [45]:
# load function from a local Python file
parq2dask = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)

parq2dask.spec.remote = True
parq2dask.spec.replicas = DASK_SHARDS 
parq2dask.spec.max_replicas = DASK_SHARDS
parq2dask.spec.service_type = 'NodePort'
parq2dask.spec.image_pull_policy = 'Always'
parq2dask.spec.build.image = IMAGE

parq2dask.export(FUNC_YAML)
# parq2dask = mlrun.import_function(FUNC_YAML)

[mlrun] 2020-02-16 14:07:28,895 function spec saved to path: /User/repos/demos/dask/yaml/parquet-to-dask.yaml


In [46]:
parq2dask.apply(mlrun.mount_v3io())
parq2dask.deploy()

# create and run the task
parq_to_dask_task = mlrun.NewTask(
    TASK_NAME, 
    handler=HANDLER,  
    params={
        'parquet_url'      : DATA_PATH,
        'sample'           : DOWNSAMPLE,
        'shards'           : DASK_SHARDS,
        'threads_per'      : DASK_THREADS_PER,
        'memory_limit'     : MEMORY_LIMIT,
        'dask_key'         : DASK_KEY,
        'target_path'      : ARTIFACTS_PATH})

# run
rn = parq2dask.run(parq_to_dask_task)

[mlrun] 2020-02-16 14:07:29,302 starting remote build, image: yjbds/mlrun-dask:dev
[mlrun] 2020-02-16 14:07:29,381 starting run user-task-parq-to-dask uid=26e5e3f6d5714ffbb9950f650165d812  -> http://mlrun-api:8080
[mlrun] 2020-02-16 14:07:29,457 saving function: parquet-to-dask, tag: latest
[mlrun] 2020-02-16 14:07:37,463 using remote dask scheduler (mlrun-parquet-to-dask-56e23aea-0) at: 18.223.249.136:30882
[mlrun] 2020-02-16 14:07:37,464 remote dashboard (node) port: 18.223.249.136:32541


/conda/lib/python3.6/site-packages/distributed/client.py:1071: VersionMismatchWarning: Mismatched versions found

lz4
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 3.0.2   |
| scheduler | 2.2.1   |
+-----------+---------+

msgpack
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 0.6.1   |
| scheduler | 0.6.2   |
+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


[mlrun] 2020-02-16 14:07:37,464 sleeping...
[mlrun] 2020-02-16 14:09:37,564 found cluster...
[mlrun] 2020-02-16 14:09:37,564 <Client: 'tcp://10.233.64.49:8786' processes=8 threads=8, memory=32.87 GB>
[mlrun] 2020-02-16 14:11:48,276 column header ['Year' 'Month' 'DayofMonth' 'DayOfWeek' 'CRSDepTime' 'UniqueCarrier'
 'ArrDelay' 'Origin' 'Dest' 'Distance']
[mlrun] 2020-02-16 14:11:49,135 log artifact scheduler at /User/repos/demos/dask/artifacts/scheduler.json, size: None, db: Y

[mlrun] 2020-02-16 14:11:49,172 run ended with state 


/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/mlrun/render.py:324: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...65d812,0,Feb 16 14:07:29,completed,user-task-parq-to-dask,kind=daskowner=adminhost=jupyter-1-56795c755c-bss6b,,parquet_url=/User/repos/demos/dask/dataset/partitionssample=0.01shards=8threads_per=8memory_limit=5GBdask_key=airlinestarget_path=/User/repos/demos/dask/artifacts,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run 26e5e3f6d5714ffbb9950f650165d812  , !mlrun logs 26e5e3f6d5714ffbb9950f650165d812 
[mlrun] 2020-02-16 14:11:49,189 run executed, status=completed


### the scheduler artifact can be shared

In [47]:
print(rn.metadata.to_yaml())

uid: 26e5e3f6d5714ffbb9950f650165d812
name: user-task-parq-to-dask
project: ''
labels:
  kind: dask
  owner: admin
  host: jupyter-1-56795c755c-bss6b
iteration: 0



In [48]:
rn.metadata.uid

'26e5e3f6d5714ffbb9950f650165d812'

In [49]:
rn.outputs['scheduler']

'/User/repos/demos/dask/artifacts/scheduler.json'

#### What's the scheduler address?

In [50]:
import json
json.load(open(rn.outputs['scheduler']))

{'type': 'Scheduler',
 'id': 'Scheduler-afb3ad89-b67f-44ee-8d61-1515b644fd85',
 'address': 'tcp://10.233.64.49:8786',
 'services': {},
 'workers': {'tcp://10.233.64.52:38467': {'type': 'Worker',
   'id': 'tcp://10.233.64.52:38467',
   'host': '10.233.64.52',
   'resources': {},
   'local_directory': '/worker-i3xwyymu',
   'name': 'tcp://10.233.64.52:38467',
   'nthreads': 1,
   'memory_limit': 4109136640,
   'last_seen': 1581862309.0022464,
   'services': {},
   'metrics': {'cpu': 2.0,
    'memory': 91942912,
    'time': 1581862308.5042937,
    'read_bytes': 285.9690461285252,
    'write_bytes': 811.9121169802883,
    'num_fds': 22,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://10.233.64.52:39118'},
  'tcp://10.233.64.53:35558': {'type': 'Worker',
   'id': 'tcp://10.233.64.53:35558',
   'host': '10.233.64.53',
   'resources': {},
   'local_directory': '/worker-m1larcx3',

### notebook cluster

Let's load the scheduler file into a cluster in this notebook using our MLRun artifact:

In [12]:
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

#client = Client(scheduler_file=rn.outputs['scheduler'])

#df = client.get_dataset(DASK_KEY)

### create a component 'on the fly' to summarise the table

The nice thing about having a dask cluster loaded with all your data is that you can write _quick and dirty_ jobs either in your notebook, a local file, or a github repo. Here we use a local file:

In [14]:
# write up function in local directory
summ = mlrun.new_function(command='/User/repos/demos/dask/code/describe.py', 
                          image=IMAGE,
                          kind='job')

summ.spec.build.image = IMAGE

summ.export('/User/repos/demos/dask/yaml/describe.yaml')

summ.apply(mlrun.mount_v3io())

summ.deploy(skip_deployed=True, with_mlrun=False)

# create the task
summ_task = mlrun.NewTask(
    'user-task-my-sum', 
    handler='table_summary',  
    params={
        'dask_key'   :  DASK_KEY,
        'dask_client': 'scheduler.json',
        'target_path':  ARTIFACTS_PATH,
        'name'       : 'table-summary.csv',
        'key'        : 'table-summary'})

# run
rn2 = summ.run(summ_task)

rn2.outputs

[mlrun] 2020-02-13 14:10:31,066 function spec saved to path: /User/repos/demos/dask/yaml/describe.yaml
[mlrun] 2020-02-13 14:10:31,074 starting run user-task-my-sum uid=ef1e3b0f7b164ff58a83328c9836d13b  -> http://mlrun-api:8080
[mlrun] 2020-02-13 14:10:31,135 Job is running in the background, pod: user-task-my-sum-24sqb
[mlrun] 2020-02-13 14:18:36,042 log artifact table-summary at /User/repos/demos/dask/artifacts/table-summary.csv, size: None, db: Y

[mlrun] 2020-02-13 14:18:36,069 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...36d13b,0,Feb 13 14:10:39,completed,describe,host=user-task-my-sum-24sqbkind=jobowner=admin,,dask_client=scheduler.jsondask_key=airlineskey=table-summaryname=table-summary.csvtarget_path=/User/repos/demos/dask/artifacts,,table-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run ef1e3b0f7b164ff58a83328c9836d13b  , !mlrun logs ef1e3b0f7b164ff58a83328c9836d13b 
[mlrun] 2020-02-13 14:18:42,707 run executed, status=completed


{'table-summary': '/User/repos/demos/dask/artifacts/table-summary.csv'}

### tests

In [16]:
import pandas as pd
pd.read_csv(rn2.outputs['table-summary'])

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,Distance
0,1.235350e+06,1.235350e+06,1.235350e+06,1.235350e+06,1.235350e+06,1.209766e+06,1.233346e+06
1,1.998624e+03,6.553828e+00,1.572554e+01,3.940275e+00,1.334899e+03,7.076394e+00,7.016191e+02
2,6.226779e+00,3.444448e+00,8.790009e+00,1.990995e+00,4.765382e+02,3.082053e+01,5.511544e+02
3,1.987000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.243000e+03,1.100000e+01
4,1.995000e+03,5.000000e+00,1.150000e+01,3.000000e+00,1.126500e+03,-3.750000e+00,4.050000e+02
5,2.003000e+03,9.000000e+00,2.000000e+01,5.000000e+00,1.530000e+03,4.500000e+00,6.995000e+02
6,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.048250e+03,8.450000e+01,2.508250e+03
7,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,1.473000e+03,4.962000e+03
